In [23]:
#!/user/bin/env python
# -*- coding:utf-8 -*-
__Author__ = 'Zijie Zhao'
__Create__ = '04/26/2022'


import os
import time
import random
import json
import argparse
import torch
import torch.nn as nn
import numpy as np
from torchvision import transforms
from torchvision import models
from PIL import Image
from torch.utils.tensorboard import SummaryWriter
from scene_seg.dataset import TrainDataset

# create a dataset and dataloader
def imresize(im, size, interp='bilinear'):
    if interp == 'nearest':
        resample = Image.NEAREST
    elif interp == 'bilinear':
        resample = Image.BILINEAR
    elif interp == 'bicubic':
        resample = Image.BICUBIC
    else:
        raise Exception('resample method undefined!')

    return im.resize(size, resample)


class BaseDataset(torch.utils.data.Dataset):
    def __init__(self, odgt, opt, **kwargs):
        # parse options
        self.imgSizes = opt['imgSizes']
        self.imgMaxSize = opt['imgMaxSize']
        # max down sampling rate of network to avoid rounding during conv or pooling
        self.padding_constant = opt['padding_constant']

        # parse the input list
        self.parse_input_list(odgt, **kwargs)

        # mean and std
        self.normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225])

    def parse_input_list(self, odgt, max_sample=-1, start_idx=-1, end_idx=-1):
        if isinstance(odgt, list):
            self.list_sample = odgt
        elif isinstance(odgt, str):
            self.list_sample = [json.loads(x.rstrip()) for x in open(odgt, 'r')]

        if max_sample > 0:
            self.list_sample = self.list_sample[0:max_sample]
        if start_idx >= 0 and end_idx >= 0:     # divide file list
            self.list_sample = self.list_sample[start_idx:end_idx]

        self.num_sample = len(self.list_sample)
        assert self.num_sample > 0
        print('# samples: {}'.format(self.num_sample))

    def img_transform(self, img):
        # 0-255 to 0-1
        img = np.float32(np.array(img)) / 255.
        img = img.transpose((2, 0, 1))
        # img = self.normalize(torch.from_numpy(img.copy()))
        img = torch.from_numpy(img.copy())
        return img

    def segm_transform(self, segm):
        # to tensor, -1 to 149
        segm = torch.from_numpy(np.array(segm)).long() - 1
        return segm

    # Round x to the nearest multiple of p and x' >= x
    def round2nearest_multiple(self, x, p):
        return ((x - 1) // p + 1) * p


class TrainDataset(BaseDataset):
    def __init__(self, root_dataset, odgt, opt, batch_per_gpu=1, **kwargs):
        super(TrainDataset, self).__init__(odgt, opt, **kwargs)
        self.root_dataset = root_dataset
        # down sampling rate of segm labe
        self.segm_downsampling_rate = opt['segm_downsampling_rate']
        self.batch_per_gpu = batch_per_gpu

        # classify images into two classes: 1. h > w and 2. h <= w
        self.batch_record_list = [[], []]

        # override dataset length when trainig with batch_per_gpu > 1
        self.cur_idx = 0
        self.if_shuffled = False

    def _get_sub_batch(self):
        while True:
            # get a sample record
            this_sample = self.list_sample[self.cur_idx]
            if this_sample['height'] > this_sample['width']:
                self.batch_record_list[0].append(this_sample) # h > w, go to 1st class
            else:
                self.batch_record_list[1].append(this_sample) # h <= w, go to 2nd class

            # update current sample pointer
            self.cur_idx += 1
            if self.cur_idx >= self.num_sample:
                self.cur_idx = 0
                np.random.shuffle(self.list_sample)

            if len(self.batch_record_list[0]) == self.batch_per_gpu:
                batch_records = self.batch_record_list[0]
                self.batch_record_list[0] = []
                break
            elif len(self.batch_record_list[1]) == self.batch_per_gpu:
                batch_records = self.batch_record_list[1]
                self.batch_record_list[1] = []
                break
        return batch_records

    def __getitem__(self, index):
        # NOTE: random shuffle for the first time. shuffle in __init__ is useless
        if not self.if_shuffled:
            np.random.seed(index)
            np.random.shuffle(self.list_sample)
            self.if_shuffled = True

        # get sub-batch candidates
        batch_records = self._get_sub_batch()

        # resize all images' short edges to the chosen size
        if isinstance(self.imgSizes, list) or isinstance(self.imgSizes, tuple):
            this_short_size = np.random.choice(self.imgSizes)
        else:
            this_short_size = self.imgSizes

        # calculate the BATCH's height and width
        # since we concat more than one samples, the batch's h and w shall be larger than EACH sample
        batch_widths = np.zeros(self.batch_per_gpu, np.int32)
        batch_heights = np.zeros(self.batch_per_gpu, np.int32)
        for i in range(self.batch_per_gpu):
            img_height, img_width = batch_records[i]['height'], batch_records[i]['width']
            this_scale = min(
                this_short_size / min(img_height, img_width), \
                self.imgMaxSize / max(img_height, img_width))
            batch_widths[i] = img_width * this_scale
            batch_heights[i] = img_height * this_scale

        # Here we must pad both input image and segmentation map to size h' and w' so that p | h' and p | w'
        batch_width = np.max(batch_widths)
        batch_height = np.max(batch_heights)
        batch_width = int(self.round2nearest_multiple(batch_width, self.padding_constant))
        batch_height = int(self.round2nearest_multiple(batch_height, self.padding_constant))

        batch_images = torch.zeros(
            self.batch_per_gpu, 3, batch_height, batch_width)
        batch_segms = torch.zeros(
            self.batch_per_gpu,
            batch_height,
            batch_width).long()

        for i in range(self.batch_per_gpu):
            this_record = batch_records[i]

            # load image and label
            image_path = os.path.join(self.root_dataset, this_record['fpath_img'])
            segm_path = os.path.join(self.root_dataset, this_record['fpath_segm'])

            img = Image.open(image_path).convert('RGB')
            segm = Image.open(segm_path)
            assert(segm.mode == "L")
            assert(img.size[0] == segm.size[0])
            assert(img.size[1] == segm.size[1])

            # random_flip
            if np.random.choice([0, 1]):
                img = img.transpose(Image.FLIP_LEFT_RIGHT)
                segm = segm.transpose(Image.FLIP_LEFT_RIGHT)

            # note that each sample within a mini batch has different scale param
            img = imresize(img, (batch_widths[i], batch_heights[i]), interp='bilinear')
            segm = imresize(segm, (batch_widths[i], batch_heights[i]), interp='nearest')

            # image transform, to torch float tensor 3xHxW
            img = self.img_transform(img)

            # segm transform, to torch long tensor HxW
            segm = self.segm_transform(segm)

            # put into batch arrays
            batch_images[i][:, :img.shape[1], :img.shape[2]] = img
            batch_segms[i][:segm.shape[0], :segm.shape[1]] = segm

        output = dict()
        output['img_data'] = batch_images
        output['seg_label'] = batch_segms

        return output

    def __len__(self):
        return int(1e10) # It's a fake length due to the trick that every loader maintains its own list

In [13]:
import torch
print(torch.__file__)

/home/software/anaconda3/2019.10_powerai/envs/powerai/lib/python3.7/site-packages/torch/__init__.py


In [19]:
!python
!import torch

Python 3.7.6 (default, Jan  8 2020, 19:13:59) 
[GCC 7.3.0] :: Anaconda, Inc. on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
KeyboardInterrupt
>>> /usr/bin/sh: import: command not found


In [21]:
!conda info --envs

# conda environments:
#
base                     /home/software/anaconda3/2019.10_powerai
Hackathon_2020_Environment_testing     /home/software/anaconda3/2019.10_powerai/envs/Hackathon_2020_Environment_testing
powerai                  /home/software/anaconda3/2019.10_powerai/envs/powerai
                         /nobackup/users/zijiezha/anaconda3
                      *  /nobackup/users/zijiezha/anaconda3/envs/opence
                         /nobackup/users/zijiezha/anaconda3/envs/py36



In [24]:
!conda activate /home/software/anaconda3/2019.10_powerai/envs/powerai


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [26]:
!conda env list

# conda environments:
#
base                     /home/software/anaconda3/2019.10_powerai
Hackathon_2020_Environment_testing     /home/software/anaconda3/2019.10_powerai/envs/Hackathon_2020_Environment_testing
powerai                  /home/software/anaconda3/2019.10_powerai/envs/powerai
                         /nobackup/users/zijiezha/anaconda3
                      *  /nobackup/users/zijiezha/anaconda3/envs/opence
                         /nobackup/users/zijiezha/anaconda3/envs/py36



In [8]:
# def createDeepLabv3(outputchannels=1):
#     """DeepLabv3 class with custom head
#     Args:
#         outputchannels (int, optional): The number of output channels
#         in your dataset masks. Defaults to 1.
#     Returns:
#         model: Returns the DeepLabv3 model with the ResNet101 backbone.
#     """
#     model = models.segmentation.deeplabv3_resnet101(pretrained=True,
#                                                     progress=True)
#     model.classifier = DeepLabHead(2048, outputchannels)
#     model.train()
#     return model


def createDeepLabv3(outputchannels=1, keep_feature_extract=False, use_pretrained=True):
    """ DeepLabV3 pretrained on a subset of COCO train2017, on the 20 categories that are present in the Pascal VOC dataset.
    """
    model_deeplabv3 = models.segmentation.deeplabv3_resnet101(pretrained=use_pretrained, progress=True)
    model_deeplabv3.aux_classifier = None
    if keep_feature_extract:
        for param in model_deeplabv3.parameters():
            param.requires_grad = False

    model_deeplabv3.classifier = models.segmentation.deeplabv3.DeepLabHead(2048, outputchannels)

    return model_deeplabv3

In [9]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.initialized = False
        self.val = None
        self.avg = None
        self.sum = None
        self.count = None

    def initialize(self, val, weight):
        self.val = val
        self.avg = val
        self.sum = val * weight
        self.count = weight
        self.initialized = True

    def update(self, val, weight=1):
        if not self.initialized:
            self.initialize(val, weight)
        else:
            self.add(val, weight)

    def add(self, val, weight):
        self.val = val
        self.sum += val * weight
        self.count += weight
        self.avg = self.sum / self.count

    def value(self):
        return self.val

    def average(self):
        return self.avg

def pixel_acc(pred, label):
        _, preds = torch.max(pred, dim=1)
        valid = (label >= 0).long()
        acc_sum = torch.sum(valid * (preds == label).long())
        pixel_sum = torch.sum(valid)
        acc = acc_sum.float() / (pixel_sum.float() + 1e-10)
        return acc

def train(segmentation_module, iterator, optimizers, criterion, history, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    ave_total_loss = AverageMeter()
    ave_acc = AverageMeter()

    segmentation_module.train()

    # main loop
    tic = time.time()
    for i in range(5000):
        # load a batch of data
        batch_data = next(iterator)[0]
        batch_data['img_data'] = batch_data['img_data'].cuda()
        batch_data['seg_label'] = batch_data['seg_label'].cuda()
        data_time.update(time.time() - tic)
        segmentation_module.zero_grad()

        # adjust learning rate
        cur_iter = i + (epoch - 1) * 5000

        # # forward pass
        pred = segmentation_module(batch_data['img_data'])['out']
        loss = criterion(pred, batch_data['seg_label'])
        acc = pixel_acc(pred, batch_data['seg_label'])

        loss = loss.mean()
        acc = acc.mean()

        # Backward
        loss.backward()
        optimizers.step()

        # measure elapsed time
        batch_time.update(time.time() - tic)
        tic = time.time()

        # update average loss and acc
        ave_total_loss.update(loss.data.item())
        ave_acc.update(acc.data.item()*100)

        # calculate accuracy, and display
        if i % 20 == 0:
            print('Epoch: [{}][{}/{}], Avg Train Time: {:.2f}, Avg Data time: {:.2f}, '
                  'Accuracy: {:4.2f}, Loss: {:.6f}'
                  .format(epoch, i, 5000,
                          batch_time.average(), data_time.average(),
                          ave_acc.average(), ave_total_loss.average()))

            fractional_epoch = epoch - 1 + 1. * i / 5000
            history['train']['epoch'].append(fractional_epoch)
            history['train']['loss'].append(loss.data.item())
            history['train']['acc'].append(acc.data.item())
          
            writer.add_scalar('training loss',
                ave_total_loss.average(),
                cur_iter)
            writer.add_scalar('training accuracy',
                ave_acc.average(),
                cur_iter)

def checkpoint(model, history, epoch):
    print('Saving checkpoints...')

    dict_model = model.state_dict()

    torch.save(
        history,
        '{}/history_epoch_{}.pth'.format(DIR, epoch))
    torch.save(
        dict_model,
        '{}/model_epoch_{}.pth'.format(DIR, epoch))

In [13]:
# class UserScatteredDataParallel(DictGatherDataParallel):
#     def scatter(self, inputs, kwargs, device_ids):
#         assert len(inputs) == 1
#         inputs = inputs[0]
#         inputs = _async_copy_stream(inputs, device_ids)
#         inputs = [[i] for i in inputs]
#         assert len(kwargs) == 0
#         kwargs = [{} for _ in range(len(inputs))]

#         return inputs, kwargs



def user_scattered_collate(batch):
        return batch

# def patch_replication_callback(data_parallel):
#     """
#     Monkey-patch an existing `DataParallel` object. Add the replication callback.
#     Useful when you have customized `DataParallel` implementation.

#     Examples:
#         > sync_bn = SynchronizedBatchNorm1d(10, eps=1e-5, affine=False)
#         > sync_bn = DataParallel(sync_bn, device_ids=[0, 1])
#         > patch_replication_callback(sync_bn)
#         # this is equivalent to
#         > sync_bn = SynchronizedBatchNorm1d(10, eps=1e-5, affine=False)
#         > sync_bn = DataParallelWithCallback(sync_bn, device_ids=[0, 1])
#     """

#     assert isinstance(data_parallel, DataParallel)

#     old_replicate = data_parallel.replicate

#     @functools.wraps(old_replicate)
#     def new_replicate(module, device_ids):
#         modules = old_replicate(module, device_ids)
#         execute_replication_callbacks(modules)
#         return modules

#     data_parallel.replicate = new_replicate

# Dataset and Loader
def main():
    gpus = [0]
    DATASET = {'root_dataset': "./data/", 
              'list_train': "./data/training.odgt",
              'list_val': "./data/validation.odgt", 
              'num_class': 150, 
              'imgSizes': (300, 375, 450, 525, 600), 
              'imgMaxSize': 1000, 
              'padding_constant': 8, 
              'segm_downsampling_rate': 8, 
              'random_flip': True}
    dataset_train = TrainDataset(
            "./data/",
            "./data/training.odgt",
            DATASET,
            batch_per_gpu=2)
    loader_train = torch.utils.data.DataLoader(
            dataset_train,
            batch_size=len(gpus),  # we have modified data_parallel
            shuffle=False,  # we do not use this param
            collate_fn=user_scattered_collate,
            num_workers=2,
            drop_last=True,
            pin_memory=True)
    print('1 Epoch = {} iters'.format(5000))

    # create loader iterator
    iterator_train = iter(loader_train)

    # load nets into gpu
    # if len(gpus) > 1:
    #     segmentation_module = UserScatteredDataParallel(
    #         segmentation_module,
    #         device_ids=gpus)
    #     # For sync bn
    #     patch_replication_callback(segmentation_module)

    # Set up optimizers
    segmentation_module = createDeepLabv3(outputchannels=150)
    optimizers = torch.optim.Adam(segmentation_module.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss(ignore_index=-1)
    segmentation_module.cuda()

    # Main loop
    history = {'train': {'epoch': [], 'loss': [], 'acc': []}}

    for epoch in range(0, 1):
        train(segmentation_module, iterator_train, optimizers, criterion, history, epoch+1)

        # checkpointing
        checkpoint(segmentation_module, history, epoch+1)

    print('Training Done!')

In [14]:
# writer = SummaryWriter('runs/ade20k-resnet50dilated-ppm_deepsup_experiment_1')
DIR = "./ckpt/ade20k-resnet50dilated-ppm_deepsup"
if not os.path.isdir(DIR):
    os.makedirs(DIR)
# parser = argparse.ArgumentParser(
#     description="PyTorch Semantic Segmentation Training"
# )

# parser.add_argument(
#     "--gpus",
#     default="0-3",
#     help="gpus to use, e.g. 0-3 or 0,1,2,3"
# )


random.seed(869)
torch.manual_seed(869)
writer = SummaryWriter('runs/ade20k-resnet50dilated-ppm_deepsup_experiment_1')
main()

# Start from checkpoint
# start_epoch = 0
# if start_epoch > 0:
#     cfg.MODEL.weights_encoder = os.path.join(
#         cfg.DIR, 'encoder_epoch_{}.pth'.format(start_epoch))
#     cfg.MODEL.weights_decoder = os.path.join(
#         cfg.DIR, 'decoder_epoch_{}.pth'.format(start_epoch))
#     assert os.path.exists(cfg.MODEL.weights_encoder) and \
#         os.path.exists(cfg.MODEL.weights_decoder), "checkpoint does not exitst!"

# # Parse gpu ids
# gpus = parse_devices(args.gpus)
# gpus = [x.replace('gpu', '') for x in gpus]
# gpus = [int(x) for x in gpus]
# num_gpus = len(gpus)
# batch_size_per_gpu = 2

# samples: 20210
1 Epoch = 5000 iters
Epoch: [1][0/5000], Time: 0.38, Data: 0.00, Accuracy: 6.80, Loss: 4.704109
Epoch: [1][20/5000], Time: 0.62, Data: 0.00, Accuracy: 21.07, Loss: 3.965649
Epoch: [1][40/5000], Time: 0.58, Data: 0.00, Accuracy: 21.47, Loss: 3.643099
Epoch: [1][60/5000], Time: 0.55, Data: 0.00, Accuracy: 23.61, Loss: 3.444899
Epoch: [1][80/5000], Time: 0.54, Data: 0.00, Accuracy: 23.47, Loss: 3.403451
Epoch: [1][100/5000], Time: 0.54, Data: 0.00, Accuracy: 24.38, Loss: 3.383046
Epoch: [1][120/5000], Time: 0.54, Data: 0.00, Accuracy: 24.97, Loss: 3.322392
Epoch: [1][140/5000], Time: 0.54, Data: 0.00, Accuracy: 25.66, Loss: 3.266722
Epoch: [1][160/5000], Time: 0.54, Data: 0.00, Accuracy: 25.54, Loss: 3.211850
Epoch: [1][180/5000], Time: 0.55, Data: 0.00, Accuracy: 25.75, Loss: 3.192372
Epoch: [1][200/5000], Time: 0.55, Data: 0.00, Accuracy: 26.44, Loss: 3.167442
Epoch: [1][220/5000], Time: 0.55, Data: 0.00, Accuracy: 26.65, Loss: 3.134791
Epoch: [1][240/5000], Time: 0.55,

In [20]:
tic = time.time()
tic

1651003465.8851838

In [21]:
tic2 = time.time()
tic2-tic

1.857238531112671

In [38]:
print('''
###############################################################

       _____                            _____            
      / ____|                          / ____|           
     | (___   ___ ___ _ __   ___ _____| (___   ___  __ _ 
      \___ \ / __/ _ \ '_ \ / _ \______\___ \ / _ \/ _` |
      ____) | (_|  __/ | | |  __/      ____) |  __/ (_| |
     |_____/ \___\___|_| |_|\___|     |_____/ \___|\__, |
                                                    __/ |
                                                   |___/                                                                    

###############################################################
Welcome to use Scene-Seg (S2)! This is a PyTorch implementation 
of semantic segmentation models on MIT ADE20K scene parsing 
dataset (http://sceneparsing.csail.mit.edu/). 

Developing this tool is also the main part of MIT 6.869 project.

-- Author: Zijie Zhao
-- Date: Apr 26 2022

''')


###############################################################

       _____                            _____            
      / ____|                          / ____|           
     | (___   ___ ___ _ __   ___ _____| (___   ___  __ _ 
      \___ \ / __/ _ \ '_ \ / _ \______\___ \ / _ \/ _` |
      ____) | (_|  __/ | | |  __/      ____) |  __/ (_| |
     |_____/ \___\___|_| |_|\___|     |_____/ \___|\__, |
                                                    __/ |
                                                   |___/                                                                    

###############################################################
Welcome to use Scene-Seg (S2)! This is a PyTorch implementation 
of semantic segmentation models on MIT ADE20K scene parsing 
dataset (http://sceneparsing.csail.mit.edu/). 

Developing this tool is also the main part of MIT 6.869 project.

-- Author: Zijie Zhao
-- Date: Apr 26 2022




In [7]:
a = 1
def foo():
    global a 
    a = a +1
    print(a)
foo()

2
